# Deep Neural Network

In [1]:
import pandas as pd
from numpy import random
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import LSTM, Conv2D, Conv1D, MaxPooling2D
from keras.layers.embeddings import Embedding
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import tensorflow as tf
import PyQt5
import matplotlib
matplotlib.use('qt4agg')
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
data = pd.read_csv(r"E:\EEG Analysis\train_molecular\label.csv", header=None)
data.head()

,0,1
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [3]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show()
        
plot_losses = PlotLosses()

In [4]:
def load_and_process(link):
    #Read data into a dataframe
    link = "E:/EEG Analysis/train_molecular/"+str(link)+".csv"
    data = pd.read_csv(link, index_col=0)
    #Convert dataframe to numpy matrix for padding
    data = data.values
    return data 

#Making a generator 
def generator(features, labels):
    #Pick a random index
    while True:
        index= random.randint(0, data.shape[0])
        #Load and preprocess the data for that index
        batch_features = load_and_process(features[index])
        batch_features = batch_features.reshape((1, 32, 1000, 1))
        #Get label for index
        batch_labels = np.array([labels[index]])
        #YEILD TO THE OVERLORDS!
        yield ( batch_features, batch_labels )

In [9]:
model = Sequential()
#model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
# model.add(LSTM(32, input_shape=(32, 1000), return_sequences=True))
model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(32, 1000, 1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 30, 998, 128)      1280      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 28, 996, 128)      147584    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 498, 128)      0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 14, 498, 128)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 14, 498, 256)      295168    
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 12, 496, 256)      590080    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 248, 256)       0         
__________

In [ ]:
model.fit_generator(generator(data[0], to_categorical(np.array(data[1]))), samples_per_epoch=data.shape[0], nb_epoch=5, verbose = 1)

In [ ]:
model.save('E:/EEG Analysis/model2.h5')

In [13]:
model = load_model('E:/EEG Analysis/model1.h5')

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_22 (LSTM)               (None, 32, 32)            132224    
_________________________________________________________________
lstm_23 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_43 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_21 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_22 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 32)                2080      
__________

In [41]:
test_data = data[[0,1]].iloc[4400:5000,:]
test_data.head()
test_data.index

RangeIndex(start=4400, stop=5000, step=1)

In [65]:
#Making a generator 
def test_generator(features, labels):
    #Pick a random index
    while True:
        index= random.randint(4400, 5000)
        #Load and preprocess the data for that index
        batch_features = load_and_process(features[index])
        batch_features = batch_features.reshape((1, 32, 1000))
        #Get label for index
        batch_labels = np.array([labels[index-4400]])
        #YEILD TO THE OVERLORDS!
        yield ( batch_features, batch_labels )

res = model.evaluate_generator(test_generator(test_data[0], np.array(test_data[1])), steps=600, verbose=1)

600/600 [==============================] - 24s 39ms/step


In [62]:
model.metrics_names

['loss', 'acc']

In [68]:
print("Accuracy is:", str(res[1]*100)[:5],"%")

Accuracy is: 73.16 %
